### Start with importing class

In [187]:
from desummation import Desummation
import numpy as np
dsm = Desummation()

### Create new matrix
$$
    A = \begin{pmatrix}
    3 & 7 & 2 & 0 \\
    -4 & 2 & 0 & -3 \\
    5 & 0 & 2 & -1 \\
    5 & -5 & -2 & -4 \\
    \end{pmatrix} 
$$

In [188]:
A = [[3, 7, 2, 0], [-4, 2, 0, -3], [5, 0, 2, -1], [5, -5, -2, -4]] 

### Now fit some random matrices to this matrix

In [189]:
dsm.fit(A, 8)

### Get all the information you might need

In [190]:
dsm.matrices()

[array([[ 0.43624987,  0.67007536,  0.82615814, -2.44571447],
        [-1.12845057,  1.32774341, -1.25423281,  0.09798059],
        [-0.28195003,  0.13298501, -1.15673484,  0.36945503],
        [ 0.20121157,  0.25330569, -1.29130113,  1.93218381]]),
 array([[-0.76110592,  1.78896002,  0.29979079,  1.03950513],
        [-0.00924252, -2.52573605,  0.55827783, -0.18640577],
        [ 0.7964691 , -0.95375585, -0.34336708,  0.6282482 ],
        [ 0.15030548,  0.66291411,  0.91736767,  1.08411935]]),
 array([[ 1.03748286e+00,  3.50633612e-01, -7.84534249e-01,
         -9.19412617e-04],
        [ 1.11393685e+00,  1.84564490e-01, -7.25198567e-01,
          5.92568347e-02],
        [ 9.05413576e-02, -8.12342851e-01,  8.86095045e-01,
         -1.13493155e-01],
        [ 1.72210919e+00,  5.74770476e-01, -1.23341277e+00,
          1.77121922e+00]]),
 array([[ 0.80293472,  0.79917473,  0.13759088,  0.08753028],
        [ 0.94502999, -0.94434594,  0.93413877,  0.70971876],
        [ 0.10118471, -1.1

In [191]:
dsm.weights()

[-0.7921641252757965,
 0.02856497634380073,
 0.8271760431198532,
 0.4419829614685806,
 0.32373200244346556,
 -3.3607544209767717,
 -2.4707198383725855,
 1.1102889503140112]

### Now let's see which matrix we got

In [192]:
dsm.predict(A)

array([[-2.61432699,  5.40926024,  1.98908676,  3.15317878],
       [-1.85441934,  4.47421948,  2.07227529, -0.47229592],
       [-1.07867   ,  6.90137977,  0.60671286, -1.49416506],
       [ 1.2140232 , -0.12356885, -2.25194497, -1.20161187]])

In [193]:
np.array(A)

array([[ 3,  7,  2,  0],
       [-4,  2,  0, -3],
       [ 5,  0,  2, -1],
       [ 5, -5, -2, -4]])

In [194]:
dsm.error()

21.635929305053782

### This is not very good approximation.
#### Certain ways to improve this:
- ##### make more trials for weights searching
- ##### make more random matrices
- ##### change metric between matrices
- ##### change distribution of elements in matrices

In [195]:
dsm.fit(A, 16, n_trials=2000, distribution='exponential', scale=1)

In [196]:
dsm.predict(A)

array([[ 2.49423493e+00,  7.03729570e+00,  1.52029639e+00,
         2.89002971e-03],
       [-4.82552532e+00,  2.56053925e+00, -2.54434113e-01,
        -4.02818656e+00],
       [ 5.33013779e+00,  7.46179565e-01,  2.69805038e+00,
        -8.14147507e-01],
       [ 3.61820494e+00, -4.10825540e+00, -2.27045590e+00,
        -2.56621879e+00]])

In [197]:
np.array(A)

array([[ 3,  7,  2,  0],
       [-4,  2,  0, -3],
       [ 5,  0,  2, -1],
       [ 5, -5, -2, -4]])

In [198]:
dsm.error()

2.9934800562620194

In [199]:
dsm.matrices()

[array([[0.77722889, 0.10296689, 0.18003875, 0.33333508],
        [1.06754551, 0.06239037, 0.26196024, 0.06821133],
        [4.23464848, 1.21672452, 0.48240398, 0.28282251],
        [0.41399084, 0.43422444, 0.78417366, 0.18906516]]),
 array([[5.21304656e+00, 1.81234538e-01, 5.44526408e-01, 4.83161307e-03],
        [7.47343942e-01, 1.15567069e+00, 4.66000964e-01, 2.55157882e-01],
        [3.86050344e-01, 1.09109111e+00, 8.10845149e-01, 1.37630101e+00],
        [1.65921920e+00, 1.00172775e+00, 1.29846076e-01, 2.33190802e+00]]),
 array([[0.29651385, 0.23469715, 1.29771109, 1.62853679],
        [0.48367742, 0.1273005 , 0.14719528, 1.31898934],
        [0.58334044, 0.82450622, 2.44264465, 4.21270427],
        [0.40553278, 1.64634184, 0.57507826, 0.21155145]]),
 array([[1.41849256, 0.2043803 , 5.64227352, 0.31583332],
        [0.55546223, 1.35292426, 0.41588722, 0.24930174],
        [2.62962855, 1.05404796, 0.86443269, 0.23213945],
        [0.65831886, 3.15212797, 0.69783266, 0.55314386]]),


In [200]:
dsm.weights()

[-0.7416757946297725,
 -2.7044970637199057,
 -6.476469954411555,
 3.8006027499983297,
 2.9322448688949017,
 1.8536921265782187,
 4.289883674097096,
 0.32352551822984577,
 0.2155499512293888,
 -2.880591723501701,
 -1.2933023480717178,
 -6.066299519204691,
 -0.888029899842083,
 -0.8156192990179143,
 4.008491664386714,
 -0.02515789902350285]

# What are the inferences?

### You can experiment with it yourself, but judging by my intuition and my experiments:
- #### Almost always we will find weights with (frobenius norm) loss close to 0 `if` we use $n^2$ weights (tested only for square matrix for now). I think that it may be some sort of `solution to a linear system`. You may understand me beforehand if you remember the basis for matrices(the one consisting of $E_{ij}$ matrices)
- #### I think a lot about applying this to ML and DL and came up to some insight: [you can see this page](https://weightagnostic.github.io/) or trust my words. The experiment showcased on this page focused on utilizing random weights in ANNs `without` any explicit `weight training`, but `rather allowing the weights between neuron connections to be learned`. This approach offers a more cost-effective training method while still achieving impressive capabilities. This just reminds me of my problem.

## Now let's get some information about loss function for some fixed basis
### Is it convex?

### First, start with simple one:
$$
A = \begin{pmatrix}
2 & -1 \\
-5 & 4 \\
\end{pmatrix}
$$


In [201]:
import plotly.graph_objects as go
dsm_convexity2 = Desummation()
A = [[2, -1], [-5, 4]]
dsm_convexity2.fit(A, 2)

### Now define a function for plotting
- #### We will need a meshgrid of x and y.
    - ##### I will be using only 2 random matrices for good visualization (x and y coordinates).
    - ##### Weights are programmed to be found from $2 \cdot min$ to $2 \cdot max$ value of matrix A, but maybe this is wrong. I don't know it yet.
- #### Also a function that will return an error, for that I will calculate the frobenius norm loss.

In [202]:
n = 200
w1 = np.linspace(-20, 20, n)
w2 = np.linspace(-20, 20, n)

In [203]:
def frobenius_loss(x, y, dsm_object, target, distance='fro'):
    B = np.tensordot(np.stack([x, y]), dsm_object.matrices(), axes=1)
    return np.linalg.norm(target - B, ord=distance)

In [204]:
def plot(w1, w2, Z):
    fig = go.Figure(data=[go.Surface(x=w1, y=w2, z=Z)], layout=go.Layout(width=600, height=400))

    # Set layout options
    fig.update_layout(
        title='3D Plot of Loss Function',
        scene=dict(
            xaxis_title='w1',
            yaxis_title='w2',
            zaxis_title='Loss',
            aspectratio=dict(x=1, y=1, z=1),
            camera=dict(
                eye=dict(x=1, y=1, z=1)
            )
        ),
        autosize=True
    )
    argmin = np.argmin(Z)
    row_index = argmin // n
    col_index = argmin % n

    print(f'Minimum value obtained at {np.min(Z)} with weights: w1:{w1[col_index]} and w2:{w2[row_index]}')
    fig.show()


In [205]:
Z = np.array([[frobenius_loss(x=x, y=y, dsm_object=dsm_convexity2, target=A) for x in w1] for y in w2])
plot(w1, w2, Z)

Minimum value obtained at 4.829581965802626 with weights: w1:1.1055276381909565 and w2:1.909547738693469


### I think it's a strike!!!
#### It's `clearly convex` function! And only with 2 weights. Case was very simple, but you can modify values in A, call dsm_convexety.fit() again to update random matrices, see if anything changes.

### Now let's try more difficult one:
#### This time convexity is quite questionable
$$
A = \begin{pmatrix}
2 & -5 & 4 \\
-3 & -3 & 2 \\
1 & 2 & 6 \\
\end{pmatrix}
$$


In [211]:
dsm_convexity3 = Desummation()
A = [[2, -5, 4], [-3, -3, 2], [1, 2, 6]]
dsm_convexity3.fit(A, 2)

In [207]:
n = 200
w1 = np.linspace(-20, 20, n)
w2 = np.linspace(-20, 20, n)

In [212]:
Z = np.array([[frobenius_loss(x=x, y=y, dsm_object=dsm_convexity3, target=A) for x in w1] for y in w2])
plot(w1, w2, Z)

Minimum value obtained at 3.945581362987292 with weights: w1:-6.130653266331658 and w2:-0.3015075376884404
